# Vision Transformer Implementation

[ViT Docs](https://huggingface.co/docs/transformers/en/model_doc/vit)

### Abstract
While the Transformer architecture has become the de-facto standard for natural language processing tasks, its applications to computer vision remain limited. In vision, attention is either applied in conjunction with convolutional networks, or used to replace certain components of convolutional networks while keeping their overall structure in place. We show that this reliance on CNNs is not necessary and a pure transformer applied directly to sequences of image patches can perform very well on image classification tasks.[[1]](#1)

![ViT Architecture](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/model_doc/vit_architecture.jpg)

## Installation

Run the following commands to install the required libraries and dependencies on the virtual environment.
```shell
conda activate env/cmil
conda install transformers datasets accelerate -c conda-forge
```


## Hugging Face Hub and Credentials

We need the hugging face read/write API token to run this notebook. Please run the cell below and paste your API key for accessing model checkpoint.

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [55]:
import os
import torch
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from datasets import Dataset, load_metric
from transformers import AutoImageProcessor, DefaultDataCollator, Trainer, TrainingArguments, AutoModelForImageClassification
from torchvision.transforms import RandomResizedCrop, ToTensor, Normalize, Compose

In [3]:
df = pd.read_csv('public/public.csv')
num_classes = ['non_globally_sclerotic_glomeruli', 'globally_sclerotic_glomeruli']
df['path'] = df.apply(lambda x: f"public/{num_classes[x['ground truth']]}/{x['name']}", axis=1)
df.head()

,name,ground truth,path
0,S-2006-005094_PAS_1of2_64552732435c92704a3d37c...,0,public/non_globally_sclerotic_glomeruli/S-2006...
1,S-2006-005094_PAS_1of2_64552732435c92704a3d37c...,0,public/non_globally_sclerotic_glomeruli/S-2006...
2,S-2006-005094_PAS_1of2_64552732435c92704a3d37c...,0,public/non_globally_sclerotic_glomeruli/S-2006...
3,S-2006-005094_PAS_1of2_64552732435c92704a3d37c...,0,public/non_globally_sclerotic_glomeruli/S-2006...
4,S-2006-005094_PAS_1of2_64552732435c92704a3d37d...,0,public/non_globally_sclerotic_glomeruli/S-2006...


In [4]:
dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda x: {'image': Image.open(x['path']).convert('RGB'), 'label': x['ground truth']}, remove_columns=['path', 'name', 'ground truth'])

Map:   0%|          | 0/5758 [00:00<?, ? examples/s]

In [7]:
dataset = dataset.train_test_split(test_size=0.2)

In [8]:
label2id, id2label = {}, {}
for i, label in enumerate(num_classes):
    label2id[label] = i
    id2label[i] = label

In [76]:
checkpoint = "google/vit-base-patch16-224-in21k"
image_size = 224
processor = AutoImageProcessor.from_pretrained(checkpoint, size=image_size)

In [77]:
_transforms = Compose([
    ToTensor(),
    Normalize(mean=processor.image_mean, std=processor.image_std)
])

def transform(examples):
    inputs = processor([image for image in examples['image']], return_tensors="pt")
    inputs['label'] = examples['label']
    return inputs

In [78]:
dataset['train'][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=788x807>,
 'label': 0}

In [79]:
ds = dataset.with_transform(transform)

In [80]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [82]:
metric = load_metric("accuracy", trust_remote_code=True)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

In [83]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(num_classes),
    id2label=id2label,
    label2id=label2id
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    remove_unused_columns=False,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    tokenizer=processor,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

In [85]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

  0%|          | 0/864 [00:00<?, ?it/s]

{'loss': 0.4518, 'grad_norm': 0.9761220812797546, 'learning_rate': 4.94212962962963e-05, 'epoch': 0.03}


  0%|          | 0/144 [00:00<?, ?it/s]

{'eval_loss': 0.3402031660079956, 'eval_accuracy': 0.8012152777777778, 'eval_runtime': 86.232, 'eval_samples_per_second': 13.359, 'eval_steps_per_second': 1.67, 'epoch': 0.03}
{'loss': 0.2895, 'grad_norm': 2.2736270427703857, 'learning_rate': 4.8842592592592595e-05, 'epoch': 0.07}


  0%|          | 0/144 [00:00<?, ?it/s]

...

{'eval_loss': 0.026823166757822037, 'eval_accuracy': 0.9930555555555556, 'eval_runtime': 41.656, 'eval_samples_per_second': 27.655, 'eval_steps_per_second': 3.457, 'epoch': 2.92}
{'loss': 0.0032, 'grad_norm': 0.023707451298832893, 'learning_rate': 8.101851851851852e-07, 'epoch': 2.95}


  0%|          | 0/144 [00:00<?, ?it/s]

{'eval_loss': 0.027684058994054794, 'eval_accuracy': 0.9930555555555556, 'eval_runtime': 40.7364, 'eval_samples_per_second': 28.279, 'eval_steps_per_second': 3.535, 'epoch': 2.95}
{'loss': 0.003, 'grad_norm': 0.02588501386344433, 'learning_rate': 2.3148148148148148e-07, 'epoch': 2.99}


  0%|          | 0/144 [00:00<?, ?it/s]

{'eval_loss': 0.027843892574310303, 'eval_accuracy': 0.9930555555555556, 'eval_runtime': 41.3565, 'eval_samples_per_second': 27.855, 'eval_steps_per_second': 3.482, 'epoch': 2.99}
{'train_runtime': 11539.1431, 'train_samples_per_second': 1.197, 'train_steps_per_second': 0.075, 'train_loss': 0.042453370087228166, 'epoch': 3.0}
***** train metrics *****
  epoch                    =         3.0
  total_flos               = 997245602GF
  train_loss               =      0.0425
  train_runtime            =  3:12:19.14
  train_samples_per_second =       1.197
  train_steps_per_second   =       0.075


In [86]:
metrics = trainer.evaluate(ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

  0%|          | 0/144 [00:00<?, ?it/s]

***** test metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.9835
  eval_loss               =     0.0603
  eval_runtime            = 0:00:42.28
  eval_samples_per_second =     27.245
  eval_steps_per_second   =      3.406


## References

<a id="1">1. Dosovitskiy, A., Beyer, L., Kolesnikov, A., Weissenborn, D., Zhai, X., Unterthiner, T., Dehghani, M., Minderer, M., Heigold, G., Gelly, S., Uszkoreit, J., & Houlsby, N. (2020). An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale. ArXiv, abs/2010.11929.</a>
